In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
#Create a Spark Application
spark = SparkSession.builder.appName("groupbyagg").getOrCreate()

In [4]:
#Load the athlete_events.csv from Local System
athlete_events = spark.read.csv('file:///home/hduser/demo_1/athlete_events.csv',sep=',',inferSchema=True,header=True)

In [5]:
athlete_events.createOrReplaceTempView("athlete_events_view")

In [8]:
#Change your DB and table name as per your requirements...
MySQL_URL = "jdbc:mysql://localhost:3306/sqoop"
MySQL_driver = "com.mysql.jdbc.Driver"
MySQL_table = "athlete_events_MySQL"
user = "root"
password = "password"

In [9]:
#Write a spark DF into a MySQL table.
athlete_events.write.format('jdbc').options(url=MySQL_URL,driver=MySQL_driver,dbtable=MySQL_table,user=user,password=password).mode('overwrite').save()

In [10]:
#Read the data from MySQL table and store it in a spark DF.
Read_From_MySQL = spark.read.format("jdbc").option("url", MySQL_URL).option("driver", MySQL_driver).option("dbtable", MySQL_table).option("user", user).option("password", password).load()


In [11]:
Read_From_MySQL.show(10)

+-----+--------------------+---+---+------+------+--------------+---+-----------+----+------+---------+----------+--------------------+-----+
|   ID|                Name|Sex|Age|Height|Weight|          Team|NOC|      Games|Year|Season|     City|     Sport|               Event|Medal|
+-----+--------------------+---+---+------+------+--------------+---+-----------+----+------+---------+----------+--------------------+-----+
|25734|Muriel Evelyn Dav...|  F| 19|   160|    50| United States|USA|1960 Summer|1960|Summer|     Roma|Gymnastics|Gymnastics Women'...|   NA|
|    1|           A Dijiang|  M| 24|   180|    80|         China|CHN|1992 Summer|1992|Summer|Barcelona|Basketball|Basketball Men's ...|   NA|
|    2|            A Lamusi|  M| 23|   170|    60|         China|CHN|2012 Summer|2012|Summer|   London|      Judo|Judo Men's Extra-...|   NA|
|    3| Gunnar Nielsen Aaby|  M| 24|    NA|    NA|       Denmark|DEN|1920 Summer|1920|Summer|Antwerpen|  Football|Football Men's Fo...|   NA|
|    4